# Guppy Demo

This notebook gives a brief introduction to Guppy.

Make sure that you have installed Guppy via `pip install guppylang`.

In [1]:
import guppylang

__name__
__doc__
__package__
__loader__
__spec__
__file__
__cached__
__builtins__
typing
ht
guppy
HSERIES_EXTENSION
MeasureCompiler
QAllocCompiler
quantum_op
angle
__name__
__doc__
__package__
__loader__
__spec__
__file__
__cached__
__builtins__
typing
ht
guppy
HSERIES_EXTENSION
MeasureCompiler
QAllocCompiler
quantum_op
angle
__name__
__doc__
__package__
__loader__
__spec__
__file__
__cached__
__builtins__
typing
ht
guppy
HSERIES_EXTENSION
MeasureCompiler
QAllocCompiler
quantum_op
angle
qubit
__name__
__doc__
__package__
__loader__
__spec__
__file__
__cached__
__builtins__
typing
ht
guppy
HSERIES_EXTENSION
MeasureCompiler
QAllocCompiler
quantum_op
angle
qubit
dirty_qubit
__name__
__doc__
__package__
__loader__
__spec__
__file__
__cached__
__builtins__
typing
ht
guppy
HSERIES_EXTENSION
MeasureCompiler
QAllocCompiler
quantum_op
angle
qubit
dirty_qubit
h
__name__
__doc__
__package__
__loader__
__spec__
__file__
__cached__
__builtins__
typing
ht
guppy
HSERIES_EXTENSION
MeasureCompiler
QAll

## Getting Started

Guppy programs are defined within a `GuppyModule`.

In [2]:
# from guppylang import GuppyModule

# module = GuppyModule(name="example_module")

Guppy functions look like regular Python functions that are annotated with the `@guppy` decorator.

In [3]:
from guppylang import guppy

@guppy
def factorial(x: int) -> int:
    """Classical factorial function in Guppy."""
    acc = 1
    while x > 0:
        acc *= x
        x -= 1
    return acc

Guppy provides a special `qubit` type that quantum operations act on. Quantum operations become available once we import and load the `guppylang.prelude.quantum` module.

In [4]:
from guppylang.prelude.quantum import qubit, h, cx, measure

# guppy.load_all(qubit, h, cx, measure)

@guppy
def bell() -> bool:
    """Prepares and measures a bell state.
    
    Returns the parity of the measurement results.
    """
    # Allocate two fresh qubits
    q1, q2 = qubit(), qubit()
    # Entangle
    q1, q2 = cx(h(q1), q2)
    q2 = h(q2)
    # Measure
    b1, b2 = measure(q1), measure(q2)
    return b1 ^ b2

Once all functions have been defined, the module can be compiled.

In [5]:
program = guppy.compile_module()
len(program.modules[0])

Guppy compilation failed. Error in file <In [4]>:12

10:    """
11:    # Allocate two fresh qubits
12:    q1, q2 = qubit(), qubit()
                ^^^^^
GuppyError: Variable `qubit` is not defined


Note that execution of programs is still work-in-progress and therefore not included in this notebook.

In [6]:
guppy._modules

{ModuleIdentifier(filename=PosixPath('/Users/seyon/ng/guppy/guppylang/prelude/builtins.py'), name='guppylang.prelude.builtins', module=<module 'guppylang.prelude.builtins' from '/Users/seyon/ng/guppy/guppylang/prelude/builtins.py'>): <guppylang.module.GuppyModule at 0x112abd410>,
 ModuleIdentifier(filename=PosixPath('/Users/seyon/ng/guppy/guppylang/prelude/angles.py'), name='guppylang.prelude.angles', module=<module 'guppylang.prelude.angles' from '/Users/seyon/ng/guppy/guppylang/prelude/angles.py'>): <guppylang.module.GuppyModule at 0x112c511d0>,
 ModuleIdentifier(filename=PosixPath('/Users/seyon/ng/guppy/guppylang/prelude/quantum.py'), name='guppylang.prelude.quantum', module=<module 'guppylang.prelude.quantum' from '/Users/seyon/ng/guppy/guppylang/prelude/quantum.py'>): <guppylang.module.GuppyModule at 0x112e3bf50>,
 ModuleIdentifier(filename=PosixPath('/var/folders/_v/lqph6hnx6dl4f164m0drzb880000gp/T/ipykernel_54228/1869001052.py'), name='1869001052.py', module=None): <guppylang.mo

## Type System

Guppy is strongly typed and requires type annotations for function arguments and returns as in the examples above.

Ill-typed programs are rejected by the compiler. For example, observe the error when trying to compile the program below.

In [6]:
bad_module = GuppyModule(name="bad_module")

@guppy(bad_module)
def bad(x: int) -> int:
    # Try to add a tuple to an int
    return x + (x, x)

bad_module.compile()  # Raises an error

Guppy compilation failed. Error in file <In [6]>:6

4:    def bad(x: int) -> int:
5:        # Try to add a tuple to an int
6:        return x + (x, x)
                 ^^^^^^^^^^
GuppyTypeError: Binary operator `+` not defined for arguments of type `int` and `(int, int)`


Furthermore, the Guppy compiler ensures that variables are definitely initialised when they are used.

In [7]:
bad_module = GuppyModule(name="bad_module")

@guppy(bad_module)
def bad(b: bool) -> int:
    if b:
        x = 4
    return x  # x not defined if b is False

bad_module.compile()  # Raises an error

Guppy compilation failed. Error in file <In [7]>:7

5:    if b:
6:        x = 4
7:    return x  # x not defined if b is False
             ^
GuppyError: Variable `x` is not defined on all control-flow paths.


Similarly, variables must have a unique type when they are used.

In [8]:
bad_module = GuppyModule(name="bad_module")

@guppy(bad_module)
def bad(b: bool) -> int:
    if b:
        x = 4
    else:
        x = True
    return int(x)  # x has different types depending on b

bad_module.compile()  # Raises an error

Guppy compilation failed. Error in file <In [8]>:9

7:    else:
8:        x = True
9:    return int(x)  # x has different types depending on b
                 ^
GuppyError: Variable `x` can refer to different types: `int` (at 6:8) vs `bool` (at 8:8)


## Linear Types

Qubits in Guppy obey *linear typing*, i.e. they cannot be copied and may only be used once before they are reassigned.

In [9]:
bad_module = GuppyModule(name="bad_module")
bad_module.load_all(guppylang.prelude.quantum)

@guppy(bad_module)
def bad(q: qubit) -> tuple[qubit, qubit]:
    return cx(q, q)

bad_module.compile()  # Raises an error

Guppy compilation failed. Error in file <In [9]>:6

4:    @guppy(bad_module)
5:    def bad(q: qubit) -> tuple[qubit, qubit]:
6:        return cx(q, q)
                       ^
GuppyError: Variable `q` with linear type `qubit` was already used (at 6:14)


Similarly, qubits cannot be implicitly dropped. They need to be returned from functions or explicitly measured or discarded.

In [10]:
bad_module = GuppyModule(name="bad_module")
bad_module.load_all(guppylang.prelude.quantum)

@guppy(bad_module)
def bad(q: qubit) -> qubit:
    tmp = h(qubit())
    tmp, q = cx(tmp, q)
    #discard(tmp)  # Compiler complains if tmp is not explicitly discarded
    return q

bad_module.compile()  # Raises an error

Guppy compilation failed. Error in file <In [10]>:7

5:    def bad(q: qubit) -> qubit:
6:        tmp = h(qubit())
7:        tmp, q = cx(tmp, q)
          ^^^
GuppyError: Variable `tmp` with linear type `qubit` is not used on all control-flow paths


## Mutual Recursion

Guppy functions support (mutual) recursion:

In [11]:
module = GuppyModule("mutual_recursion")

@guppy(module)
def is_even(x: int) -> bool:
    if x == 0:
        return True
    return is_odd(x-1)

@guppy(module)
def is_odd(x: int) -> bool:
    if x == 0:
        return False
    return is_even(x-1)

program = module.compile()

## Structs

Structs can be used to group quantum and classical data.

In [12]:
module = GuppyModule("structs")
module.load_all(guppylang.prelude.quantum)

@guppy.struct(module)
class QubitPair:
    q1: qubit
    q2: qubit

    @guppy(module)
    def method(self: "QubitPair") -> "QubitPair":
        self.q1 = h(self.q1)
        self.q1, self.q2 = cx(self.q1, self.q2)
        return self

@guppy(module)
def make_struct() -> QubitPair:
    pair = QubitPair(qubit(), qubit())
    # pair = pair.method()  # TODO: Calling methods doesn't work yet
    return pair

program = module.compile()

## Nested & Higher-Order Functions

Guppy supports nested function definitions with variables captured from outer scopes.

In [13]:
module = GuppyModule("nested_function")

@guppy(module)
def outer(x: int) -> int:
    def nested(y: int) -> int:
        return x + y
    return nested(42)

program = module.compile()

However, similar to regular Python, variables from enclosing functions scopes may not be modified. Python's `global` and `nonlocal` statements are not supported by Guppy.

In [14]:
bad_module = GuppyModule(name="bad_module")

@guppy(bad_module)
def outer(x: int) -> int:
    def nested() -> None:
        x += 1  # Mutation of captured variable x is not allowed
    nested()
    return x

bad_module.compile()  # Raises an error

Guppy compilation failed. Error in file <In [14]>:6

4:    def outer(x: int) -> int:
5:        def nested() -> None:
6:            x += 1  # Mutation of captured variable x is not allowed
              ^^^^^^
GuppyError: Variable `x` defined in an outer scope (at 4:10) may not be assigned to


Functions can be used as higher-order values using Python's `Callable` type.

In [15]:
from typing import Callable

module = GuppyModule("higher_order")

@guppy(module)
def increment_by(inc: int) -> Callable[[int], int]:
    def closure(x: int) -> int:
        return x + inc
    return closure

@guppy(module)
def main(x: int) -> int:
    inc5 = increment_by(5)
    return inc5(x)

program = module.compile()